In [1]:
import h5py
import numpy as np
import sys
sys.path.append('../')
sys.path.append('/home/amber/multitask_RNA/DNA_BERT_rep/')
import utils
import torch
from transformers import RobertaForMaskedLM,FeatureExtractionPipeline
from dna_tokenizer import DNATokenizer
import os
from tqdm import tqdm
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [2]:
model = RobertaForMaskedLM.from_pretrained('/home/amber/multitask_RNA/DNA_BERT_rep/small-roberta-lr8/checkpoint-23500/',
                                            output_hidden_states = True).to('cuda')
file = h5py.File('../../data/rna_stable/insert_dataset.h5','r')
tokenizer = DNATokenizer('/home/amber/multitask_RNA/DNA_BERT_rep/vocab.txt')

In [3]:
roberta_output = h5py.File('../../data/rna_stable/roberta_embed.h5','w')
batch_size = 32
for dataset in ['test','train','valid']:
    key = 'X_'+dataset
    onehot = file[key]
    string_seq = utils.onehot_to_seq(onehot)
    split_seq = []
    for seq in string_seq:
        split_seq.append(' '.join([seq[i:i+6] for i in range(0, len(seq)-6+1, 1)]))
    
    token_seq = tokenizer.batch_encode_plus(split_seq,max_length=507)
    output_cache = []
    for seq_i in tqdm(range(0,len(token_seq['input_ids']),batch_size)):
        seq_batch = torch.tensor(token_seq['input_ids'][seq_i:seq_i+batch_size]).to('cuda')
        output_seq = model(seq_batch)[-1][-1].cpu().detach().numpy()
        output_cache.extend(output_seq)
    roberta_output.create_dataset(name=key,data = np.array(output_cache))
    roberta_output.create_dataset(name='Y_'+dataset,data = file['Y_'+dataset][:])

100%|██████████| 36/36 [00:01<00:00, 28.58it/s]


In [21]:
roberta_output.keys()

<KeysViewHDF5 ['X_test', 'X_train', 'X_valid', 'Y_test', 'Y_train', 'Y_valid']>

In [22]:
roberta_output.close()